# Constructor for buildings Units Base

# Algorithm
1. loop every building in script file and in future script file
2. create a list of number of original units
3. set the number of buidling
4. set the number of Iteration (if the file will be appneded with the rest of the buildings
5. set the size of appartments (according to the average that was decided upon)
6. set the price according to the data
7. set the buy price according to the data
8. set number of floors according to data
9. set status "old buidling"
10. save the file as a seperate Excel file (in the future this will be one file)


## loading dictionary and libraries

In [1093]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import os
import ipywidgets as widgets
import numpy as np

In [1094]:
#reference dictionary file
path_1 = 'library.json'
with open(path_1) as json_file:
    data = json.load(json_file)

json_formatted_str = json.dumps(data, indent=2)
print(json_formatted_str)

#rules dictionary file file
path_2 = 'rules.json'
with open(path_2) as json_file:
    rules_1 = json.load(json_file)


json_formatted_str = json.dumps(rules_1, indent=2)
print('-----Rules File-------')
print(json_formatted_str)

{
  "GIS_package": {
    "old_bldgs": {
      "package": "Source/data/GIS/BaseLayers.gpkg",
      "layer": "bldgs_renewal_2020"
    }
  },
  "script_file": "Source/data/script/script_ssi_march_8_535pm.xlsx",
  "future_script": "Source/data/FutureBldgs_Tama_38.xlsx",
  "bldg_path": "Source/data/bldg/",
  "Agents": "Source/Agents_8pm.xlsx",
  "Google Drive Prepath": "content/drive/My Drive/BatYam NY DRIVE/Simulations/March 4 2020 After NYC/"
}
-----Rules File-------
{
  "Maintenance": {
    "old_bldgs": {
      "4": "100",
      "8": "180",
      "12": "200"
    },
    "new_bldgs": {
      "4": "250",
      "8": "320",
      "12": "400",
      "15": "450"
    }
  },
  "Income": {
    "Low": {
      "0": "10000"
    },
    "Medium": {
      "1": "18000"
    },
    "High": {
      "2": "100000"
    }
  },
  "ArnonaTaxExplanation": "(65.23/12.0)",
  "ArnonaTax": "5.4358"
}


## loading script files

In [1095]:
script = pd.read_excel(data['script_file'])
script_ft =  pd.read_excel(data['future_script'])

In [1096]:
script

,index,bld_address,OriginalUnits,OriginalFloors,OriginalHouseSize,purchase_p,rent_price,Maintenace,Old_Purchase,OldRent,...,bld_operation,pointPolygon,ParcelChange,FuturePlanID,ExcelBefore,ExcelChange,ExcelPolygon,GIS_files,Address Title,TypeTitle
0,0,215_22,18,3.0,88,22930,62,100,2017840,5556,...,1,Point,No,20140882,before_215_22.xlsx,after_215_22.xlsx,NaN,bldgs_1a,"סוקולוב 22, בת ים",תמ״א 38/1
1,1,216_10,12,3.0,123,24376,68,100,2998248,8464,...,1,Point,No,20120398,before_216_10.xlsx,after_216_10.xlsx,NaN,bldgs_2a,"מסריק 10, בת ים",תמ״א 38/1
2,2,216_4,12,3.0,94,23351,67,100,2194994,6398,...,1,Point,No,20141086,before_216_4.xlsx,after_216_4.xlsx,NaN,bldgs_3a,"מסריק 4, בת ים",תמ״א 38/1
3,3,209_6,6,3.0,94,22922,50,100,2154668,4800,...,2,Point,No,20140231,before_209_6.xlsx,after_209_6.xlsx,NaN,bldgs_6a,"ארלוזורוב 9, בת ים",תמ״א 38/2
4,4,complex_4,46,2.5,72,21336,62,100,1536192,4564,...,3,Polygon,Yes,502-0317495,before_complex_4.xlsx,after_complex_4.xlsx,polygon_502_0317495.xlsx,bldgs_45a,"הרצל 16,18 וחנה סנש 11, 9, 7, בת ים",תב״ע פינוי בינוי


In [1097]:
script_ft

,index,bld_address,OriginalUnits,OriginalFloors,OriginalHouseSize,purchase_p,rent_price,addFloors,addUnits,addArea,priceIncrease,rentIncrease,TotalUnits,TotalFloors,AvrgTotaalArea,newMaintenace,Purchase,Rent,bld_operation
0,0,215_22,18,3,88,22930,62,4,15,12,0.10,0.1,33,7,100,180,2522300.0,7000.0,1
1,1,216_10,12,3,123,24376,68,2,8,12,0.10,0.1,20,5,135,180,3619836.0,10278.0,1
2,2,216_4,12,3,94,23351,67,3,10,12,0.10,0.1,22,6,106,180,2722726.6,7992.2,1
3,3,209_6,6,3,94,22922,50,0,0,25,0.15,0.2,25,9,119,320,3273261.6,7460.0,2


In [1098]:
Agents = pd.DataFrame()

## initating iteration

In [1099]:
agents_name = widgets.Text(value='agents_0309_9_55pm.xlsx')
display(agents_name)

Text(value='agents_0309_9_55pm.xlsx')

In [1100]:

i_slider = widgets.IntSlider(min=0, max =6)
display(i_slider)

IntSlider(value=0, max=6)

In [1103]:
i

0

In [1107]:
i = i_slider.value
currentBldg = script.loc[i]
if (i != 0):
    path ="Source/"+agents_name.value
    Agents = pd.read_excel(path)
    Agents = Agents[['AgentAppUnit', 'AgentBldAdd', 'AgentIncome', 'AgentWealth','AgentPurchaseThreshold', 'AgentRentThreshold', 'Age',
       'OwnerShip', 'AgeGroup', 'Native_Seniority', 'Native_Group', 'AgentID']]

else:
        Agents = pd.DataFrame()
if (currentBldg.bld_operation < 3):
    current_bldg_ds = pd.DataFrame()
    Future_bldg_ds = pd.DataFrame()
    subAgents_ds = pd.DataFrame()
    
    #original building

    
    cb = def_currentBld_op12_ds(currentBldg)
    bldg_path = data['bldg_path']
    sp_bldg_path = bldg_path + currentBldg.ExcelBefore
    cb.to_excel(sp_bldg_path)
    
    #Future building or construction
    futureBldg = script_ft.loc[i]
    fb = def_futureBld_op12_ds(futureBldg)
    bldg_path = data['bldg_path']
    sp_bldg_path = bldg_path + currentBldg.ExcelChange
    fb.to_excel(sp_bldg_path)
    
    #Update Agents
    ng = generateAgensts(cb)
    Agents = Agents.append(ng)
    Agents.reset_index(inplace=True)
    Agents['AgentID'] = Agents.index
    Agents = Agents[['AgentAppUnit', 'AgentBldAdd', 'AgentIncome', 'AgentWealth','AgentPurchaseThreshold', 'AgentRentThreshold', 'Age',
           'OwnerShip', 'AgeGroup', 'Native_Seniority', 'Native_Group', 'AgentID']]
    path ="Source/"+agents_name.value
    Agents.to_excel(path)
    
else:
    print('oh oh operation 3!')
    #operation 3 means that it's a Polygon not a single building
    get_complex_xlsx = currentBldg.ExcelPolygon
    bldg_path = data['bldg_path']
    bldg_path_xlsx= bldg_path+get_complex_xlsx
    inside_script = pd.read_excel(bldg_path_xlsx)
    before_dem = inside_script[inside_script.status=='destroyed']
    after_dem = inside_script[inside_script.status=='New Building']



oh oh operation 3!


In [1108]:
if (currentBldg.bld_operation < 3):
    print(currentBldg.bld_operation)
#     current_bldg_ds['appUnits'] = unitNumber
#     current_bldg_ds['bldAdd'] = currentBldg.bld_address
#     current_bldg_ds['AppSize'] = currentBldg.OriginalHouseSize
#     current_bldg_ds['sellPerMeter'] = currentBldg.purchase_p
#     current_bldg_ds['rentPerMeter'] = currentBldg.rent_price
#     current_bldg_ds['sellPrice'] = current_bldg_ds['AppSize'] * current_bldg_ds['sellPerMeter']
#     current_bldg_ds['rentPerMonth'] = current_bldg_ds['AppSize'] * current_bldg_ds['rentPerMeter'] + currentBldg.Maintenace
#     current_bldg_ds['maintenace'] = currentBldg.Maintenace
#     if (currentBldg.bld_operation == 1):
#         current_bldg_ds['status'] = 'Old Building'
#     else:
#         current_bldg_ds['status'] = 'Demolished'
else:
    print(before_dem.index.tolist())
    after_dem.reset_index(inplace=True)
    print(after_dem.index.tolist())
    print('Operation 3')
    last_line1 = len(before_dem.index)-1
    last_line2 = len(after_dem.index)-1

[0, 1, 2, 3, 4]
[0, 1]
Operation 3


In [1109]:
j_slider = widgets.IntSlider(min=0, max =last_line1)
display(j_slider)

IntSlider(value=0, max=4)

In [1115]:
j = j_slider.value 

if (j==0):
    current_bldg_ds = pd.DataFrame()
    #Future_bldg_ds = pd.DataFrame()
    subAgents_ds = pd.DataFrame()
    #after_dem.reset_index(inplace=True)
before_line = before_dem.loc[j]
current_bldg_ds = current_bldg_ds.append(def_currentBld_op3_ds(before_line))

if (j==last_line1):
    print('last line!')
    bldg_path = data['bldg_path']
    sp_bldg_path = bldg_path + currentBldg.ExcelBefore
    current_bldg_ds.reset_index(inplace=True)
    current_bldg_ds.to_excel(sp_bldg_path)
    subAgents_ds = generateAgensts(current_bldg_ds)

    Agents = Agents.append(subAgents_ds)
    Agents.reset_index(inplace=True)
    Agents['AgentID'] = Agents.index
    Agents = Agents[['AgentAppUnit', 'AgentBldAdd', 'AgentIncome', 'AgentWealth','AgentPurchaseThreshold', 'AgentRentThreshold', 'Age',
           'OwnerShip', 'AgeGroup', 'Native_Seniority', 'Native_Group', 'AgentID']]
    path ="Source/"+agents_name.value
    Agents.to_excel(path)
#after_line = after_dem.loc[j]
#Future_bldg_ds = Future_bldg_ds.append(def_FutureBld_op3_ds(after_line))


last line!


In [1116]:
last_line2 = len(after_dem.index)-1
print(last_line2)
#after_dem.reset_index(inplace=True)

1


In [1118]:
j_slider = widgets.IntSlider(min=0, max =last_line2)
display(j_slider)




IntSlider(value=0, max=1)

In [1120]:
j=j_slider.value
if (j==0):
    Future_bldg_ds = pd.DataFrame()
after_line = after_dem.loc[j]
Future_bldg_ds = Future_bldg_ds.append(def_FutureBld_op3_ds(after_line))
if (j==1):
    bldg_path = data['bldg_path']
    sp_bldg_path = bldg_path + currentBldg.ExcelChange
    Future_bldg_ds.reset_index(inplace=True)
    Future_bldg_ds.to_excel(sp_bldg_path)


In [983]:
def def_currentBld_op12_ds(this_line):
    ds = pd.DataFrame()
    
    nUnits = this_line.OriginalUnits
    unitNumber = [k for k in range(nUnits)]
    
    ds['appUnits'] = unitNumber
    ds['bldAdd'] = this_line.bld_address
    ds['AppSize'] = this_line.OriginalHouseSize
    ds['sellPerMeter'] = this_line.purchase_p
    ds['rentPerMeter'] = this_line.rent_price
    
    ds['ArnonaTarif'] = float(rules_1['ArnonaTax'])
    ds['ArnonaTax'] = ds['ArnonaTarif'] * ds['AppSize']
    
    ds['sellPrice'] = ds['AppSize'] * ds['sellPerMeter']
    ds['rentPerMonth'] = ds['AppSize'] * ds['rentPerMeter'] + this_line.Maintenace +  ds['ArnonaTax']
    ds['maintenace&Tax'] = this_line.Maintenace + ds['ArnonaTax']
    
    
    if (this_line.bld_operation == 1):
        ds['status'] = 'Old Building'
    else:
        ds['status'] = 'Demolished'
    return ds

In [984]:
def def_futureBld_op12_ds(this_line):
    ds = pd.DataFrame()
    nUnits = this_line.TotalUnits
    unitNewNumber = [k for k in range(nUnits)]
    ds['appUnits'] = unitNewNumber
    ds['bldAdd'] = this_line.bld_address
    ds['AppSize'] = this_line.AvrgTotaalArea
    ds['sellPerMeter'] = this_line.purchase_p * ( 1 + this_line.priceIncrease )
    ds['rentPerMeter'] = this_line.rent_price * ( 1 + this_line.rentIncrease )
    
    ds['ArnonaTarif'] = float(rules_1['ArnonaTax'])
    ds['ArnonaTax'] = ds['ArnonaTarif'] * ds['AppSize']
    
    ds['sellPrice'] = ds['AppSize'] * ds['sellPerMeter']
    ds['rentPerMonth'] = ds['AppSize'] * ds['rentPerMeter'] + this_line.newMaintenace + ds['ArnonaTax'] 
    
    ds['newMaintenace&Tax'] = this_line.newMaintenace + ds['ArnonaTax']
    
    if (currentBldg.bld_operation ==1 ):
        ds['status'] = 'Tama 38_1'
    else:
        ds['status'] = 'Tama 38_2'
    return ds

In [985]:
def def_currentBld_op3_ds(before_line):
    ds = pd.DataFrame()
    
    nUnits = before_line.OriginalUnits
    unitNumber = [k for k in range(nUnits)]
    
    ds['appUnits'] = unitNumber
    ds['bldAdd'] = before_line.bld_address
    ds['AppSize'] = before_line.OriginalHouseSize
    ds['sellPerMeter'] = before_line.purchase_p
    ds['rentPerMeter'] = before_line.rent_price

    ds['ArnonaTarif'] = float(rules_1['ArnonaTax'])
    ds['ArnonaTax'] = ds['ArnonaTarif'] * ds['AppSize']
    
    ds['sellPrice'] = ds['AppSize'] * ds['sellPerMeter']
    ds['rentPerMonth'] = ds['AppSize'] * ds['rentPerMeter'] + before_line.Maintenace +  ds['ArnonaTax']
    ds['maintenace&Tax'] = before_line.Maintenace +  ds['ArnonaTax']
    ds['status'] = 'Demolished'
    return ds

In [986]:
def def_FutureBld_op3_ds(after_line):
    ds = pd.DataFrame()
    nUnits = int(after_line.TotalUnits)
    unitNewNumber = [k for k in range(nUnits)]
    ds['appUnits'] = unitNewNumber
    ds['bldAdd'] = after_line.bld_address
    ds['AppSize'] = after_line.AvrgTotaalArea
    ds['sellPerMeter'] = after_line.purchase_p * ( 1 + after_line.priceIncrease )
    ds['rentPerMeter'] = after_line.rent_price * ( 1 + after_line.rentIncrease )

    
    ds['ArnonaTarif'] = float(rules_1['ArnonaTax'])
    ds['ArnonaTax'] = ds['ArnonaTarif'] * ds['AppSize']
    
    ds['sellPrice'] = ds['AppSize'] * ds['sellPerMeter']
    ds['rentPerMonth'] = ds['AppSize'] * ds['rentPerMeter'] + after_line.newMaintenace + ds['ArnonaTax'] 
    
    ds['newMaintenace&Tax'] = after_line.newMaintenace + ds['ArnonaTax'] 
    ds['status'] = 'New Building'
    return ds
        

In [1056]:
def generateAgensts(thisDS):
    nUnits = len(thisDS['appUnits'])
    #unitNumber = [k for k in range(nUnits)]
    
    rand_income_rent = np.random.randint(0, high=5000, size=nUnits, dtype='l') #monthly income
    rand_inc_sr_rent =  pd.Series(rand_income_rent)

    rand_income_own = np.random.randint(3000, high=15000, size=nUnits, dtype='l') #monthly income
    rand_inc_sr_own =  pd.Series(rand_income_own)
    
    rand_wealth = np.random.randint(0, high=5000000, size=nUnits, dtype='l') #wealth
    age = np.random.randint(25, high=110, size=nUnits, dtype='l') #wealth
    ownership = np.random.randint(0, high=2, size=nUnits, dtype='l') #0 rent 1 owns
    
    ds = pd.DataFrame()
    
    ds['OwnerShip'] = pd.Series(ownership)
    ds['AgentAppUnit'] = thisDS['appUnits'] #what unit does the agent live in
    ds['AgentBldAdd'] = thisDS['bldAdd'] # in what bld address does the Agent live in
    
    #ds['AgentIncome'] = (thisDS['rentPerMonth'])/0.3 + pd.Series(rand_income) #Income for One
    
    #ds['AgentIncome'] = (thisDS['rentPerMonth'])/0.3 + pd.Series(rand_income) #Income for One
 
    ds['AgentWealth'] = pd.Series(rand_wealth)
    
    ds['AgentPurchaseThreshold'] = ds['AgentWealth'] * 0.3 # 30% of wealth, Agent can buy a house without Mortgage
    ds['Age'] = pd.Series(age)
    
    ds['AgeGroup'] = ds['Age'].apply(lambda x: 0 if x < 65 else 1)
    ds['Native_Seniority'] = ds['Age'].apply(getRandomSeniority)
    ds['Native_Group'] = ds['Native_Seniority'].apply(lambda x: 0 if x < 10 else 1) # 0 Native, 1 NewComer
    
    # for renters take the rent price
    ds.loc[ds['OwnerShip']==0,'AgentIncome'] = (thisDS[ds['OwnerShip']==0]['rentPerMonth'])/0.3 + rand_inc_sr_rent[ds['OwnerShip']==0]
    ds.loc[ds['OwnerShip']==1,'AgentIncome'] = (thisDS[ds['OwnerShip']==1]['maintenace&Tax'])/0.3 + rand_inc_sr_own[ds['OwnerShip']==1]
    ds['AgentRentThreshold'] = ds['AgentIncome'] * 0.3 # 30% of income
    
    return ds



In [993]:
def getRandomSeniority(age):
    max_seniority = 50
    min_seniority = 10
    if age > 50:
        max_seniority = 50
        min_seniority = 10
    else:
        max_seniority = age
        min_seniority = 10
    return np.random.randint(min_seniority, high=max_seniority, size=1, dtype='l')[0]
        

# Data Formating
doing some formating to the Agents file to make it more clear for statistical purposes.

In [1121]:
Agents = Agents[['AgentAppUnit', 'AgentBldAdd', 'AgentIncome', 'AgentWealth','AgentPurchaseThreshold', 'AgentRentThreshold', 'Age',
           'OwnerShip', 'AgeGroup', 'Native_Seniority', 'Native_Group', 'AgentID']]
path ="Source/"+agents_name.value
#Agents.to_excel(path)

In [1130]:
Agents.tail(5)

,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
89,7,211_11,21620,4253837,1276151.1,6486.1282,36,0,0,32,1,89,Young,Native,Rent,2,High Income
90,8,211_11,23541,2575160,772548.0,7062.4282,67,0,1,42,1,90,Old,Native,Rent,2,High Income
91,9,211_11,23490,4736181,1420854.3,7047.1282,39,0,0,33,1,91,Young,Native,Rent,2,High Income
92,10,211_11,22139,3070365,921109.5,6641.8282,76,0,1,16,1,92,Old,Native,Rent,2,High Income
93,11,211_11,7527,1961759,588527.7,2258.3282,102,1,1,42,1,93,Old,Native,Owner,0,Low Income


In [1124]:
Agents['AgentIncome'] = Agents['AgentIncome'].astype(int)

In [1126]:
#Age Formating
Agents['AgeGroupDesc'] = Agents['Age'].apply(lambda x: 'Young' if x < 65 else 'Old')
#Native Formating
Agents['NativeDesc'] = Agents['Native_Group'].apply(lambda x: 'Native' if x == 1 else 'New Comers')
#Ownership Formating
Agents['OwnershipDesc'] = Agents['OwnerShip'].apply(lambda x: 'Owner' if x == 1 else 'Rent')

In [1127]:
LowInc = int(rules_1['Income']['Low']['0'])
MediumInc = int(rules_1['Income']['Medium']['1'])
HighInc = int(rules_1['Income']['High']['2'])
Agents['IncomeNum'] = Agents['AgentIncome'].apply(lambda x: "0" if x < LowInc else "1" if x < MediumInc else "2")
Agents['IncomeDesc'] = Agents['AgentIncome'].apply(lambda x: 'Low Income' if x < LowInc else 'Medium Income' if x < MediumInc else "High Income")

In [1132]:
Agents.to_excel(path)

In [1133]:
Agents.tail()

,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
89,7,211_11,21620,4253837,1276151.1,6486.1282,36,0,0,32,1,89,Young,Native,Rent,2,High Income
90,8,211_11,23541,2575160,772548.0,7062.4282,67,0,1,42,1,90,Old,Native,Rent,2,High Income
91,9,211_11,23490,4736181,1420854.3,7047.1282,39,0,0,33,1,91,Young,Native,Rent,2,High Income
92,10,211_11,22139,3070365,921109.5,6641.8282,76,0,1,16,1,92,Old,Native,Rent,2,High Income
93,11,211_11,7527,1961759,588527.7,2258.3282,102,1,1,42,1,93,Old,Native,Owner,0,Low Income
